In [42]:
import itertools #iteration tools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # nice plots
import matplotlib.pyplot as plt # plots
from ipy_table import *# dem tables!

#preprocessing
from sklearn.preprocessing import LabelEncoder

# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/train.csv')
test_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/test.csv')
both = [train_df, test_df]
train_df.head()
super_table=[]
super_table.append(['name', 'reg_function', 'trainset_acc', 'testset_acc', 'r2score', 'conf_matrx', 'kaggle_acc'])
df_load = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv')
for row in df_load.values.tolist():
    super_table.append(row)

In [5]:
for df in both:
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Don', 'Dr', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major'], 'Officer')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [50]:
#labelling
var_mod = ['Title', 'Sex', 'Ticket', 'Embarked']
le = LabelEncoder()
for i in var_mod:
    mask = ~train_df[i].isnull()
    train_df[i][mask] = le.fit_transform(train_df[i][mask])
    mask2 = ~test_df[i].isnull()
    test_df[i][mask2] = le.fit_transform(test_df[i][mask2])
train_df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Embarked      int64
Title        object
dtype: object

In [7]:
train_df = train_df.drop(['Cabin', 'Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Cabin', 'Name'], axis=1)

In [8]:
#perform imputation!
#imputation of age with imputer
from sklearn.preprocessing import Imputer
imp=Imputer(missing_values="NaN", strategy="mean" )
train_df["Age"]=imp.fit_transform(train_df[["Age"]]).ravel()
test_df["Age"]=imp.fit_transform(test_df[["Age"]]).ravel()

#impute embarked as the most frequent one
freq_port = train_df.Embarked.dropna().mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)

#impute missing Fare value
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [46]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,0,3,1,22.0,1,0,523,7.2500,2,2
1,1,1,0,38.0,1,0,596,71.2833,0,3
2,1,3,0,26.0,0,0,669,7.9250,2,1
3,1,1,0,35.0,1,0,49,53.1000,2,3
4,0,3,1,35.0,0,0,472,8.0500,2,2


In [11]:
#Split data for train and test
def split_data(data, target='Survived'):
    #set target for training
    target = train_df["Survived"]
    target.head(3)

    # Import the train_test_split method
    from sklearn.model_selection import train_test_split
    # Split data into train (2/3rd of data) and test (1/3rd of data)
    return train_test_split(train_df, target, train_size = 0.75, random_state=0);

In [12]:
def regr_equation(logreg, train, target):
    if type(model) is LogisticRegression:
        coef = logreg.coef_[0]
        intercept = "{:.2f}".format(logreg.intercept_[0])
    else:
        coef = logreg.coef_
        intercept = "{:.2f}".format(logreg.intercept_)
        
    output = target.name + ' = ' + str(intercept) + ' + '
    coeff_df = pd.DataFrame(train.columns.delete(0))
    coeff_df.columns = ['Feature']
    coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
    features = coeff_df['Feature'].tolist()
    coefficients = coeff_df['Correlation'].tolist()
    
    for coeff, feature in zip(coefficients, features):
        coeff_str = "{:.2f}".format(coeff)
        output += coeff_str + "*" + str(feature) + " + "
    return output[:-3]

In [13]:
def confusion_matrix(model, X, y):
    from sklearn.metrics import confusion_matrix
    return confusion_matrix(y, model.predict(X))

In [65]:
def test_model (model, data, submission_name = None, test=None):
   
    if test is None:
        
        from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
        train, test, target_train, target_test = split_data(data)
        model.fit(train, target_train)
        #Calc parameters
        if type(model) is LogisticRegression:
            function_str = regr_equation(model, train, target_train)
        elif type(model) is LinearRegression:
            function_str = regr_equation(model, train, target_train)
        else :
            function_str = "NA"    

        if type(model) is not LinearRegression:
            trainset_acc = round(accuracy_score(target_train, model.predict(train)) * 100, 2)
            testset_acc = round(accuracy_score(target_test, model.predict(test)) * 100, 2)
            conf_matrix = confusion_matrix(target_train, model.predict(train))
            conf_matrix = 'TN: '+str(conf_matrix[0][0])+', FP: '+str(conf_matrix[0][1])+ \
                              ', FN: '+str(conf_matrix[1][0])+', TP: '+str(conf_matrix[1][1])
        else:
            trainset_acc = 'NA'
            testset_acc = 'NA'
            conf_matrix = "NA"

        r2_score = r2_score(target_train, model.predict(train))
        kaggle = "not_tested"

        #prints
        print("-"*40)
        print('Submission name:', submission_name )
        print('Regression function:\n', function_str)
        print('Accuracy on train set:', trainset_acc)
        print('Accuracy on test set:', testset_acc)
        print("R2 score:", r2_score)
        print("Confusion matrix:\n", conf_matrix)
        return [submission_name, function_str, str(trainset_acc), str(testset_acc), r2_score, conf_matrix, kaggle]
    else:
        
        train = data.drop("Survived", axis=1)
        target_train = data["Survived"]
        test  = test_df.drop("PassengerId", axis=1).copy()
        model.fit(train, target_train)
        pred = model.predict(test)
        if type(model) is XGBClassifier:
            predictions = [round(value) for value in pred]
        submission = pd.DataFrame({
                "PassengerId": test_df["PassengerId"],
                "Survived": pred
            })
        submission.to_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/output/'+submission_name+'.csv',
                          index=False)

In [15]:
#Setup
suffix = '_basicImputation_Title_newFun'

modelLogReg = LogisticRegression()
modelLinReg = LinearRegression()
modelSVC = SVC()
modellinSVC = LinearSVC()
modelKN = KNeighborsClassifier(n_neighbors = 3)
modelGNB = GaussianNB()
modelPercp = Perceptron()
modelSGD = SGDClassifier()
modelTree = DecisionTreeClassifier()
modelRndForest = RandomForestClassifier(n_estimators=100)

models = {modelLogReg: "LogisticRegr", modelLinReg: "LinearRegr",
          modelSVC: "SVC", modellinSVC: "LinSVC",
          modelKN: "KN", modelGNB: "GaussianNB",
          modelPercp: "Percep", modelSGD: "SGD",
          modelTree: "DecisionTree", modelRndForest: "RandomForest"}

In [43]:
xgboostmodel = xgboost.XGBClassifier()
models = {xgboostmodel : "XGBoost"}

In [64]:
train_df['Sex']=train_df['Sex'].apply(int)
train_df['Title']=train_df['Title'].apply(int)
train_df['Ticket']=train_df['Ticket'].apply(int)

test_df['Sex']=test_df['Sex'].apply(int)
test_df['Embarked']=test_df['Embarked'].apply(int)
test_df["Ticket"] = test_df["Ticket"].apply(int)
test_df['Title']=test_df['Title'].apply(int)

In [60]:
outputs = []
for model, prefix in models.items():
    name=prefix+suffix
    outputs.append(test_model(model, train_df, submission_name=name))
outputs.sort(key=lambda x: x[0])

----------------------------------------
('Submission name:', 'XGBoost_basicImputation_Title_newFun')
('Regression function:\n', 'NA')
('Accuracy on train set:', 100.0)
('Accuracy on test set:', 100.0)
('R2 score:', 1.0)
('Confusion matrix:\n', 'TN: 410, FP: 0, FN: 0, TP: 258')


In [ ]:
models = {modelLogReg: "LogisticRegr",
          modellinSVC: "LinSVC",
          modelGNB: "GaussianNB",
          modelTree: "DecisionTree", 
          modelRndForest: "RandomForest"}

In [62]:
models = {xgboostmodel : "XGBoost"}

In [66]:
for model, prefix in models.items():
    name=prefix+suffix
    test_model(model, train_df, submission_name=name, test=test_df)

In [68]:
outputs[]

[['XGBoost_basicImputation_Title_newFun',
  'NA',
  '100.0',
  '100.0',
  1.0,
  'TN: 410, FP: 0, FN: 0, TP: 258',
  'not_tested']]

In [18]:
for out in outputs:
    super_table.append(out)

In [40]:
make_table(super_table)
apply_theme('basic')

name,reg_function,trainset_acc,testset_acc,r2score,conf_matrx,kaggle_acc
logisticReg_basic,Survived = 3.99 + -0.85*Pclass + -0.00*Name + -2.35*Sex + -0.03*Age + -0.32*SibSp + -0.06*Parch + -0.00*Ticket + 0.01*Fare,79.9700,nan,0.1696,"TN: 369, FP: 55, FN: 88, TP: 202",failed_evl
linearReg_basic,Survived = 1.40 + -0.19*Pclass + -0.00*Name + -0.49*Sex + -0.01*Age + -0.06*SibSp + -0.01*Parch + -0.00*Ticket + 0.00*Fare,40.2500,nan,0.4025,nan,failed_evl
SVC_basic,nan,99.8600,nan,0.9942,"TN: 424, FP: 0, FN: 1, TP: 289",failed_evl
KNC_basic,nan,82.4900,nan,0.2742,"TN: 356, FP: 68, FN: 57, TP: 233",failed_evl
GaussianNB_basic,nan,78.5700,nan,0.1116,"TN: 352, FP: 72, FN: 81, TP: 209",failed_evl
Perceptron_basic,nan,62.4600,nan,-0.5562,"TN: 417, FP: 7, FN: 261, TP: 29",failed_evl
LinearSVC_basic,nan,49.4400,nan,-1.0962,"TN: 94, FP: 330, FN: 31, TP: 259",failed_evl
SGDClassifier_basic,nan,65.6900,nan,-0.4227,"TN: 330, FP: 94, FN: 151, TP: 139",failed_evl
DecisionTreeClassifier_basic,nan,100.0000,nan,1.0000,"TN: 424, FP: 0, FN: 0, TP: 290",failed_evl
RandomForestClassifier_basic,nan,100.0000,nan,1.0000,"TN: 424, FP: 0, FN: 0, TP: 290",failed_evl


In [47]:
df = pd.DataFrame(super_table[1:], columns=super_table[0])
df.to_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv', 
                      index=False)